In [1]:
%matplotlib notebook
from __future__ import division

/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
from matplotlib.pyplot import plot, ylabel, xlabel, yscale, xscale, legend, subplots, gca, gcf
from matplotlib.ticker import FormatStrFormatter
from theano import function
import numpy as np
import gzip
import cPickle
from scipy.optimize import minimize
from climin.util import optimizer
from itertools import repeat, cycle, islice, izip
import random
inf = float("inf")

gnumpy: failed to import cudamat. Using npmat instead. No GPU will be used.


In [3]:
from breze.learn.data import one_hot
from breze.learn.base import cast_array_to_local_type
from schlichtanders.myfunctools import compose, meanmap, summap, compose_fmap, Average
from schlichtanders import myfunctools
from schlichtanders.mygenerators import eatN, chunk, chunk_list, every, takeN
from schlichtanders.myplot import add_val, add_point

In [4]:
from theano_models import (Merge, Flatten, Reparameterize, reduce_all_identities,
                           inputting_references, outputting_references)
from theano_models.tools import (as_tensor_variable, total_size, clone, clone_all,PooledRandomStreams,
                                 get_profile, squareplus, squareplus_inv, softplus, softplus_inv)
from theano_models.visualization import d3viz
from IPython.display import IFrame
import theano_models.deterministic_models as dm
import theano_models.probabilistic_models as pm
import theano_models.extra_models as em
import theano_models.postmaps as post
from theano_models.composing import normalizing_flow, variational_bayes

In [5]:
import theano
import theano.tensor as T
from theano.printing import debugprint
from theano.tensor.shared_randomstreams import RandomStreams

In [6]:
from sqlalchemy import Column, Integer, Unicode, UnicodeText, String, PickleType, Float
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base

In [7]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [8]:
inputting_references.update(['to_be_randomized'])
inputting_references, outputting_references

({'flat',
  'inputs',
  'n_data',
  'noise',
  'parameters',
  'parameters_positive',
  'parameters_psumto1',
  'to_be_randomized'},
 {'kl_prior', 'logP', 'loglikelihood', 'norm_det', 'outputs'})

In [9]:
from schlichtanders.myobjects import NestedNamespace

In [10]:
pm.RNG = NestedNamespace(PooledRandomStreams(pool_size=int(5e8)), RandomStreams())

# Data

In [11]:
datafile = '../data/mnist.pkl.gz'
# Load data.        a                                                                                           

with gzip.open(datafile,'rb') as f:                                                                        
    train_set, val_set, test_set = cPickle.load(f)                                                       

X, Z = train_set                                                                                               
VX, VZ = val_set
TX, TZ = test_set

Z = one_hot(Z, 10)
VZ = one_hot(VZ, 10)
TZ = one_hot(TZ, 10)

# from UncertainWeightsPaper rescaled the data,
# however it already seems to be normalized
# X /= 126  
# VX /= 126
# TX /= 126
# X *= 2
# VX *= 2
# TX *= 2

image_dims = 28, 28

X, Z, VX, VZ, TX, TZ = [cast_array_to_local_type(i) for i in (X, Z, VX,VZ, TX, TZ)]
map(np.shape, [X, Z, VX, VZ, TX, TZ])

[(50000, 784),
 (50000, 10),
 (10000, 784),
 (10000, 10),
 (10000, 784),
 (10000, 10)]

# Hyperparameters

In [12]:
engine = create_engine('sqlite:///hyperparameters.db')
Base = declarative_base(bind=engine)

class RandomHyper(Base):
    __tablename__ = 'hyper'
    id = Column(Integer, primary_key=True)
    
    # hyper parameters:
    n_epochs = Column(Integer)
    batch_size = Column(Integer)
    mapreduce = Column(String)
    average_n = Column(Integer)
    units_per_layer = Column(Integer)
    pi = Column(Float)
    minus_log_s1 = Column(Integer)
    minus_log_s2 = Column(Integer)
    
    n_normflows = Column(Integer)
    
    opt_identifier = Column(String)
    opt_momentum = Column(Float)
    opt_offset = Column(Float)
    opt_decay = Column(Float)
    opt_step_rate = Column(Float)
    
    # results:
    best_val_loss = Column(Float)
    best_parameters = Column(PickleType, nullable=True)
    train_loss = Column(PickleType)
    val_loss = Column(PickleType)

    def __init__(self):
        # hyper parameters:
        self.n_epochs = 20
        self.batch_size = 128
        self.mapreduce = random.choice(["summap", "meanmap"])
        self.average_n = 1
        self.units_per_layer = np.random.choice([400, 800, 1200], p=[0.5, 0.3, 0.2])
        self.pi = random.choice([1/4, 1/2, 3/4])
        self.minus_log_s1 = random.choice([0,1,2])
        self.minus_log_s2 = random.choice([6,7,8])
        
        self.n_normflows = random.choice([1,2,3,4,8,32])
        
        self.opt_identifier = random.choice(["adadelta", "adam", "rmsprop"])
        if self.opt_identifier == "adadelta":
            self.opt_momentum = random.choice([np.random.uniform(0, 0.01), np.random.uniform(0.9, 1)])
            self.opt_offset = random.choice([5e-5, 1e-8])
            self.opt_step_rate = random.choice([1, 1e-3, 1e-4, 1e-5])
        elif self.opt_identifier == "adam":
            self.opt_momentum = random.choice([np.random.uniform(0, 0.01), np.random.uniform(0.8, 0.93)])
            self.opt_offset = 10 ** -np.random.uniform(3, 4)
            self.opt_step_rate = random.choice([1e-3, 1e-4, 1e-5])
        elif self.opt_identifier == "rmsprop":
            self.opt_momentum = random.choice([np.random.uniform(0.002, 0.008), np.random.uniform(0.9, 1)])
            self.opt_offset = np.random.uniform(0, 0.000045)
            self.opt_step_rate = random.choice([1e-3, 1e-4, 1e-5])
        self.opt_decay = np.random.uniform(0.78, 1)
        
        self.init_results()
    
    def init_results(self):
        # extra for being able to reset results for loaded hyperparameters
        self.best_parameters = None
        self.best_val_loss = inf
        self.train_loss = []
        self.val_loss = []

Base.metadata.create_all()
Session = sessionmaker(bind=engine)
sql_session = Session()
hyper = RandomHyper()
hyper.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x7f50957c5d90>,
 'average_n': 1,
 'batch_size': 128,
 'best_parameters': None,
 'best_val_loss': inf,
 'mapreduce': 'summap',
 'minus_log_s1': 0,
 'minus_log_s2': 6,
 'n_epochs': 20,
 'n_normflows': 4,
 'opt_decay': 0.8047322013410626,
 'opt_identifier': 'rmsprop',
 'opt_momentum': 0.007177364176257046,
 'opt_offset': 4.326533864677926e-05,
 'opt_step_rate': 0.0001,
 'pi': 0.75,
 'train_loss': [],
 'units_per_layer': 800,
 'val_loss': []}

# Model

## data modelling

In [13]:
# this is extremely useful to tell everything the default sizes
input = as_tensor_variable(X[0], name="X")

In [ ]:
predictor = dm.Mlp(
    input=input,
    output_size=Z.shape[1],
    output_transfer="softmax",
    hidden_sizes=[hyper.units_per_layer]*2,
    hidden_transfers=["rectifier"]*2
)
predictor

In [ ]:
target_distribution = pm.Categorical(predictor)
target_distribution

In [ ]:
targets = Merge(target_distribution, predictor,
                Flatten(predictor['parameters'], flat_key="to_be_randomized")) #givens={predictor['inputs'][0]: X[0]}
targets

## parameter modelling

In [ ]:
params_base = pm.DiagGauss(output_size=total_size(targets['to_be_randomized']))  
# if you want to use size directly, CAUTION, you need to copy before!
# params_base.map('parameters_positive', reparameterize_map(squareplus, squareplus_inv), 'parameters')
params_base

In [ ]:
normflows = [dm.PlanarTransform() for _ in range(hyper.n_normflows)]
normflows

In [ ]:
params = params_base
for transform in normflows:
    params = normalizing_flow(transform, params)  # returns transform, however with adapted logP    

params

## bayes

In [ ]:
g1 = pm.Gauss(total_size(targets['to_be_randomized']), init_var=np.exp(-2* hyper.minus_log_s1))
g2 = pm.Gauss(total_size(targets['to_be_randomized']), init_var=np.exp(-2* hyper.minus_log_s2))
prior = pm.Mixture(g1, g2, mixture_probs=[hyper.pi, 1-hyper.pi])
# label hyper parameters accordingly
prior = Merge(prior,
              parameters=None, # mean is not adapted at all, but left centred at zero
              parameters_positive='hyperparameters_positive',
              parameters_psumto1='hyperparameters_psumto1')
prior

In [ ]:
model = variational_bayes(targets, 'to_be_randomized', params, priors=prior)
model

In [ ]:
# model = Merge(model, Reparameterize(model['parameters_positive'], squareplus, squareplus_inv))
# UncertainWeights uses softplus parameterization
model = Merge(model, Reparameterize(model['parameters_positive'], softplus, softplus_inv))
model

In [ ]:
model = Merge(model, Flatten(model['parameters']))
#               em.NoisePool(model['noise']) #, givens={predictor['inputs'][0]: X[0]}
model

# Optimizer

In [ ]:
# dm.InvertibleModel.INVERTIBLE_MODELS
reduce_all_identities()
# dm.InvertibleModel.INVERTIBLE_MODELS

Climin wants an iterator of (args, kwarsg) as keyword argument "args" (to be passed to the loss function). Concretley, we use an infinite iterator over minibatches.

In [ ]:
n_batches = X.shape[0] // hyper.batch_size  # after this many steps we went through the whole data set once

climin_args = izip(izip(chunk(hyper.batch_size, cycle(Z)), chunk(hyper.batch_size, cycle(X))), repeat({}))

def weights_regularizer_1epoch():
    for i in range(1, n_batches+1):
        yield 2**(n_batches - i) / (2**n_batches - 1)
        
assert len(list(weights_regularizer_1epoch())) == n_batches

In [ ]:
mapreduce = getattr(myfunctools, hyper.mapreduce)
if hyper.average_n > 1:
    mapreduce = compose_fmap(Average(hyper.average_n), mapreduce)

postmap = compose(post.flat_numericalize_postmap, post.variational_postmap) 
postmap_kwargs = {
    'mapreduce': mapreduce,  # TODO add more functionality for composed fmaps, with args
    'annealing_combiner': post.AnnealingCombiner(
        weights_regularizer=cycle(weights_regularizer_1epoch())
    ),
    'adapt_init_params': lambda ps: ps + np.random.normal(size=ps.size, scale=0.01),
#     'profile': True,
    'mode': 'FAST_RUN'
}
optimizer_kwargs = postmap(model, **postmap_kwargs)
climin_kwargs = post.climin_postmap(optimizer_kwargs)
climin_kwargs

In [ ]:
opt = optimizer(
    identifier=hyper.opt_identifier,
    step_rate=hyper.opt_step_rate,
    momentum=hyper.opt_momentum,
    decay=hyper.opt_decay,
    offset=hyper.opt_offset,
    
    args=climin_args,
    **climin_kwargs
)

# Visualized Fit

In [ ]:
# plot: 
line_train, = plot([], [], 'go-', label="average training loss")
line_curr_val, = plot([],[], 'bo:', label="avrg current validation loss")
line_best_val, = plot([], [], 'ko-', label="avrg best validation loss")
# plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
yscale('log')
gca().yaxis.set_minor_formatter(FormatStrFormatter("%.2g"))
ylabel("validation loss")
xlabel("#epoch")
legend(loc='lower left', fancybox=True, framealpha=0.5)

# start values:
val_loss = optimizer_kwargs['num_loss'](optimizer_kwargs['num_parameters'], VZ, VX, no_annealing=True)
training_loss = optimizer_kwargs['num_loss'](optimizer_kwargs['num_parameters'], Z[:10], X[:10], no_annealing=True)
start_epoch = opt.n_iter//n_batches

add_point(line_best_val, start_epoch, val_loss)
add_point(line_curr_val, start_epoch, val_loss)
add_point(line_train, start_epoch, training_loss)

In [ ]:
for info in takeN(hyper.n_epochs, every(n_batches, opt)):
    current_epoch = info['n_iter']//n_batches
    # collect and visualize validation loss for choosing the best model
    val_loss = optimizer_kwargs['num_loss'](opt.wrt, VZ, VX, no_annealing=True)
    if val_loss < hyper.best_val_loss:
        hyper.best_parameters = opt.wrt
        hyper.best_val_loss = val_loss
        add_point(line_best_val, current_epoch, val_loss)
    hyper.val_loss.append(val_loss)
    add_point(line_curr_val, current_epoch, val_loss)
    
    # visualize training loss for comparison:
    training_loss = optimizer_kwargs['num_loss'](opt.wrt, Z[:10], X[:10], no_annealing=True)
    hyper.train_loss.append(training_loss)
    add_point(line_train, current_epoch, training_loss)
    
#     print info['n_iter'], training_loss, val_loss, best_val_loss

In [ ]:
sql_session.add(hyper)
sql_session.commit()

# Performance

TODO: average over predictions

In [ ]:
print best_val_loss
model['flat'] = best_wrt

predict = mlp.function()
predict(X[0]), Z[0]

In [ ]:
PX = np.apply_along_axis(predict, 1, X)
PVX = np.apply_along_axis(predict, 1, VX)
PTX = np.apply_along_axis(predict, 1, TX)

In [ ]:
print 'incorrect samples train/val/test:  %i/%i/%i' % (
    (PX[:, :10].argmax(1) != Z.argmax(1)).sum(),
    (PVX[:, :10].argmax(1) != VZ.argmax(1)).sum(),
    (PTX[:, :10].argmax(1) != TZ.argmax(1)).sum())

print 'error rate train/val/test:  %g/%g/%g' % (
    (PX[:, :10].argmax(1) != Z.argmax(1)).mean(),
    (PVX[:, :10].argmax(1) != VZ.argmax(1)).mean(),
    (PTX[:, :10].argmax(1) != TZ.argmax(1)).mean())